## Instructions from Guided Capstone<a id='1.1_Imports'></a>

Goal: Create a cleaned development dataset you can use to complete the modeling step of your project.  <br>
    ● Create dummy or indicator features for categorical variables Hint: you’ll need to think about your old favorite pandas functions here like get_dummies() . <br>
    ● Standardize the magnitude of numeric features using a scaler <br>
    
Review the following questions and apply them to your dataset: <br>
    ● Does my data set have any categorical data, such as Gender or day of the week? <br>
    ● Do my features have data values that range from 0 - 100 or 0-1 or both and more? <br>

## 1.1 General Imports<a id='1.1_Imports'></a>


In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2.1 Data<a id='1.2_Data'></a>

In [675]:
ped = pd.read_csv('ped_eda.csv')
bike = pd.read_csv('bike_eda.csv')
nc_cities = pd.read_csv('nc_cities_eda.csv')

ped = ped.drop(columns = 'Unnamed: 0')
bike = bike.drop(columns = 'Unnamed: 0')
nc_cities = nc_cities.drop(columns = 'Unnamed: 0')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (17,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(GMC) I've decided I'm going to try to use this capstone as ane exercise for supervised learning. The labels here will be the injury type. Can we use some of the numerous features already present to predict the label of injury type (e.g. No Injuries, minor injuries, killed, etc.)?

There are lot of things that I could do, and there could be some deeper or more meaningful insights that could be possible, but that is not my goal for this project.

(GMC) The first thing I'm going to need to do now that I have a goal (which is to do supervised learning), I need to prepare my data. The data as I prepared them last time has many null values in them, which at the time I didn't want to remove to 'preserve' the data. But, since I am going to be using ML, I need to get rid of them.

## 2.2. Data Cleaning Second Pass

(GMC) But before I do that, I need to decide what features I want to use to predict my labels

In [676]:
ped.columns

Index(['X', 'Y', 'OBJECTID', 'AmbulanceR', 'City', 'County', 'CrashAlcoh',
       'CrashDay', 'CrashGrp', 'CrashHour', 'CrashLoc', 'CrashMonth',
       'CrashSevr', 'CrashType', 'CrashYear', 'Developmen', 'DrvrAge',
       'DrvrAgeGrp', 'DrvrAlcDrg', 'DrvrAlcFlg', 'DrvrInjury', 'DrvrRace',
       'DrvrSex', 'DrvrVehTyp', 'HitRun', 'Latitude', 'LightCond', 'Locality',
       'Longitude', 'NumLanes', 'NumUnits', 'PedAge', 'PedAgeGrp', 'PedAlcDrg',
       'PedAlcFlag', 'PedInjury', 'PedPos', 'PedRace', 'PedSex', 'RdCharacte',
       'RdClass', 'RdConditio', 'RdConfig', 'RdDefects', 'RdFeature',
       'RdSurface', 'Region', 'RuralUrban', 'SpeedLimit', 'TraffCntrl',
       'Weather', 'Workzone'],
      dtype='object')

In [677]:
ped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33707 entries, 0 to 33706
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   X           33707 non-null  float64
 1   Y           33707 non-null  float64
 2   OBJECTID    33706 non-null  float64
 3   AmbulanceR  33707 non-null  object 
 4   City        33707 non-null  object 
 5   County      33707 non-null  object 
 6   CrashAlcoh  33707 non-null  object 
 7   CrashDay    33706 non-null  object 
 8   CrashGrp    33153 non-null  object 
 9   CrashHour   33707 non-null  float64
 10  CrashLoc    33661 non-null  object 
 11  CrashMonth  33707 non-null  object 
 12  CrashSevr   33345 non-null  object 
 13  CrashType   33707 non-null  object 
 14  CrashYear   33707 non-null  float64
 15  Developmen  33707 non-null  object 
 16  DrvrAge     27629 non-null  object 
 17  DrvrAgeGrp  27629 non-null  object 
 18  DrvrAlcDrg  20780 non-null  object 
 19  DrvrAlcFlg  27034 non-nul

(GMC) Desirable Fields <br>

City Size (Could make this categorical such as big/small city OR min/max scaled by population) <br>
AmbulanceR (maybe) <br>
CrashAlcoh (binary of whether or not alcohol was involved) <br>
CrashMonth (month the crash occurred) <br>
CrashDay (day of the week of the crash) <br>
CrashHour (hour of the day the crash occurred) <br>
CrashSevr (Typeof Injury - This is the label category) <br>
DrvrAgeGrp (The DrvrAge has a 70+ category, which makes it difficult to use, so I will use this one instead) <br>
DrvrInjury (Type of Driver Injury. May exclude) <br>
DrvrVehType (This has many categories, but would like an approximate size of vehicle needs cleaning) <br>
HitRun (Binary) <br>
LightCond (couple of categories to clean up) <br>
PedAgeGrp (tSame with DrvrAge, has 70+ cat. Going to use provided grouping) <br>
PedAlcDrg <br>
RdClass (type of road e.g. Local Street, US Route) <br>
SpeedLimit <br>
TraffCntrl (many categries. Mostly just want to know if there was any kind of control, Easily collapsable) <br>
Weather <br>

(GMC) Bike has similar fields but Bike in place of the name of Ped


### Ped

In [678]:
ped2 = ped[['City', 'AmbulanceR', 'DrvrAlcDrg', 'CrashMonth', 'CrashDay', 
            'CrashHour', 'CrashSevr', 'DrvrAgeGrp', 'DrvrInjury', 'DrvrVehTyp',
            'HitRun', 'LightCond', 'PedAgeGrp', 'PedAlcDrg', 'RdClass', 'SpeedLimit', 'TraffCntrl', 'Weather']]

In [679]:
ped2.CrashMonth.value_counts()

October      3575
November     3318
December     3164
September    3005
August       2727
May          2668
March        2654
April        2632
January      2608
June         2469
February     2446
July         2441
Name: CrashMonth, dtype: int64

In [680]:
# Probably an automated/faster way to do this
months = {'January': 1,
         'February': 2,
         'March' : 3,
         'April' : 4,
         'May': 5,
         'June': 6,
         'July': 7,
         'August': 8,
         'September': 9,
         'October' : 10,
         'November' : 11,
         'December': 12}

new_values = [months[item] for item in list(ped2.CrashMonth)]

In [681]:
ped2.loc[:,'CrashMonth'] = pd.Series(new_values).copy()
# I get a warning, but it works
ped2.CrashMonth.head()

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


0    1
1    1
2    1
3    1
4    1
Name: CrashMonth, dtype: int64

(GMC) CrashDay is missing 1 value. It's also missing the CrashSevr. I'm just going to drop it.

In [682]:
ped2[ped2.CrashDay.isnull()]

,City,AmbulanceR,DrvrAlcDrg,CrashMonth,CrashDay,CrashHour,CrashSevr,DrvrAgeGrp,DrvrInjury,DrvrVehTyp,HitRun,LightCond,PedAgeGrp,PedAlcDrg,RdClass,SpeedLimit,TraffCntrl,Weather
24225,Winston-Salem,No,No,1,NaN,11.0,NaN,60-69,O: No Injury,Passenger Car,No,Daylight,70+,No,Public Vehicular Area,NaN,No Control Present,Cloudy


In [683]:
ped2 = ped2[~ped2.CrashDay.isnull()]
ped2.shape

(33706, 18)

In [684]:
# Map days of the week to ordinal values
days = {'Sunday' : 1,
        'Monday' : 2,
        'Tuesday' : 3,
        'Wednesday' : 4,
        'Thursday' : 5,
        'Friday' : 6,
        'Saturday' : 7}

new_values = [days[item] for item in list(ped2.CrashDay)]
new_values[:5]

[7, 4, 4, 5, 6]

In [685]:
ped2.CrashDay = pd.Series(new_values).astype('int64')

In [686]:
ped2.CrashDay.value_counts()

6.0    5601
5.0    5029
4.0    4984
3.0    4935
7.0    4888
2.0    4742
1.0    3526
Name: CrashDay, dtype: int64

(GMC) The vehicle types have a few nulls, but the types are dominated by passenger cars. I will fill the nulls with that value. Then, I will categorize each type as small or large.

In [687]:
ped2.DrvrVehTyp = ped2.DrvrVehTyp.fillna('Passenger Car')

In [688]:
cars = {'Passenger Car' : 'small',
        'Sport Utility' : 'small',
        'Pickup' : 'small',
        'Van' : 'small',
        'Light Truck (Mini-Van, Panel)': 'small',
        'Single Unit Truck (2-Axle, 6-Tire)' : 'large',
        'Tractor/Semi-Trailer' : 'large',
        'Motorcycle' : 'small',
        'Police' : 'small',
        'Commercial Bus' : 'large',
        'Truck/Trailer' : 'small', #I'm assuming this is a pickup with a trailer
        'Single Unit Truck (3 Or More Axles)' : 'large',
        'Taxicab' : 'small',
        'School Bus': 'large',
        'Other Bus': 'large',
       'Motor Home/Recreational Vehicle' : 'large',
       'Unknown Heavy Truck': 'large',
       'EMS Vehicle, Ambulance, Rescue Squad': 'small',
       'Moped' : 'small',
       'Truck/Tractor': 'large',
       'Tractor/Doubles': 'large',
       'Firetruck' : 'large',
       'Other': 'small',
        'All Terrain Vehicle (ATV)' : 'small',
       'Activity Bus': 'small',
       'Pedestrian': 'small',
       'Farm Tractor': 'large',
       'Autocycle': 'small',
       'Motor Scooter Or Motor Bike Or Motor': 'small'}

new_values = [cars[item] for item in list(ped2.DrvrVehTyp)]

In [689]:
ped2.loc[:,'DrvrVehTyp'] = pd.Series(new_values).copy()

In [690]:
ped2.loc[:,'DrvrVehTyp'] = ped2.DrvrVehTyp.fillna('small')

In [691]:
ped2.DrvrVehTyp.value_counts()

small    32940
large      766
Name: DrvrVehTyp, dtype: int64

(GMC) Dealing with lighting, there are some null values, but there are CrashHours available for most of them.

In [692]:
ped2.LightCond.value_counts()

Daylight                      18726
Dark - Lighted Roadway         7479
Dark - Roadway Not Lighted     5742
Dusk                            936
Dawn                            506
Dark - Unknown Lighting         202
Other                            48
Name: LightCond, dtype: int64

In [693]:
ped2.loc[ped2.LightCond.isnull() & ((ped2.CrashHour >= 8) & (ped2.CrashHour <= 16)), 'LightCond'] = 'Daylight'
ped2.loc[ped2.LightCond.isnull(), 'LightCond'] = 'Dark - Unknown Lighting'

In [694]:
ped2.LightCond.isnull().value_counts()

False    33706
Name: LightCond, dtype: int64

(GMC) Pedestrian Age groups are a total mess.

In [695]:
# Default is a mess
ped2.PedAgeGrp.value_counts()

30-39    4075
40-49    3877
50-59    3728
20-24    3137
25-29    2771
16-19    2173
60-69    2158
11-15    1873
70+      1445
41-50    1225
31-40    1120
6-10     1058
0-5       890
16-20     861
51-60     849
21-25     821
26-30     648
61-79     382
71+       357
Name: PedAgeGrp, dtype: int64

In [696]:
# Let's merge some of these groupings
ped2.loc[ped2['PedAgeGrp'] == '71+', 'PedAgeGrp'] = '70+'
ped2.loc[ped2['PedAgeGrp'] == '16-20', 'PedAgeGrp'] = '16-19'
ped2.loc[ped2['PedAgeGrp'] == '21-25', 'PedAgeGrp'] = '20-24'
ped2.loc[ped2['PedAgeGrp'] == '26-30', 'PedAgeGrp'] = '25-29'
ped2.loc[ped2['PedAgeGrp'] == '31-40', 'PedAgeGrp'] = '30-39'
ped2.loc[ped2['PedAgeGrp'] == '41-50', 'PedAgeGrp'] = '40-49'
ped2.loc[ped2['PedAgeGrp'] == '51-60', 'PedAgeGrp'] = '50-59'
ped2.loc[ped2['PedAgeGrp'] == '61-79', 'PedAgeGrp'] = '60-69'

In [697]:
# Consolidated
ped2.PedAgeGrp.value_counts()

30-39    5195
40-49    5102
50-59    4577
20-24    3958
25-29    3419
16-19    3034
60-69    2540
11-15    1873
70+      1802
6-10     1058
0-5       890
Name: PedAgeGrp, dtype: int64

In [698]:
ped2['PedAgeGrp'].isnull().value_counts()

False    33448
True       258
Name: PedAgeGrp, dtype: int64

In [699]:
# Not all hit and runs. Will probably fill in with most common value
ped2[ped2['PedAgeGrp'].isnull()].head()

,City,AmbulanceR,DrvrAlcDrg,CrashMonth,CrashDay,CrashHour,CrashSevr,DrvrAgeGrp,DrvrInjury,DrvrVehTyp,HitRun,LightCond,PedAgeGrp,PedAlcDrg,RdClass,SpeedLimit,TraffCntrl,Weather
59,None - Rural Crash,No,NaN,1,5.0,15.0,O: No Injury,25-29,O: No Injury,small,No,Daylight,NaN,NaN,US Route,50 - 55 MPH,No Control Present,Clear
68,Chapel Hill,No,NaN,1,7.0,18.0,C: Possible Injury,30-39,O: No Injury,small,No,Dark - Roadway Not Lighted,NaN,NaN,Local Street,30 - 35 MPH,No Control Present,Clear
97,Greensboro,Yes,NaN,1,4.0,10.0,C: Possible Injury,40-49,O: No Injury,small,No,Daylight,NaN,NaN,Public Vehicular Area,NaN,No Control Present,Clear
138,Charlotte,No,NaN,1,7.0,7.0,A: Suspected Serious Injury,40-49,O: No Injury,small,No,Daylight,NaN,NaN,Local Street,30 - 35 MPH,No Control Present,Clear
264,Chapel Hill,No,NaN,2,3.0,9.0,C: Possible Injury,NaN,NaN,small,Yes,Daylight,NaN,NaN,Local Street,30 - 35 MPH,No Control Present,Rain


In [700]:
ped2.loc[:,'PedAgeGrp'] = ped2.PedAgeGrp.fillna('30-39')

In [701]:
# I think I am fine with this as is
# may want to merge the routes together.
ped2.RdClass.value_counts()

Local Street              16604
Public Vehicular Area      8639
State Secondary Route      3269
US Route                   1984
NC Route                   1716
Private Road, Driveway      908
Interstate                  586
Name: RdClass, dtype: int64

In [702]:
ped2.RdClass.isnull().value_counts()

False    33706
Name: RdClass, dtype: int64

In [703]:
# This is fine as is.
ped2.SpeedLimit.value_counts()

30 - 35  MPH    10544
40 - 45  MPH     5579
5 - 15 MPH       5544
20 - 25  MPH     4611
50 - 55  MPH     3137
60 - 75 MPH       635
Name: SpeedLimit, dtype: int64

In [704]:
ped2.SpeedLimit.isnull().value_counts()

False    30050
True      3656
Name: SpeedLimit, dtype: int64

In [705]:
ped2[ped2.SpeedLimit.isnull()]['RdClass'].value_counts()

Public Vehicular Area     2567
Local Street               652
Private Road, Driveway     309
US Route                    52
State Secondary Route       43
NC Route                    27
Interstate                   6
Name: RdClass, dtype: int64

In [706]:
# I'm going to use the most common speed limits for the road classes to manually imput the nulls
most_common = {}
road_types = [road for road in ped2.RdClass.value_counts().index]

for road_type in road_types:
    most_common[road_type] = ped2[ped2.RdClass == road_type]['SpeedLimit'].value_counts().index[0]

most_common = pd.Series(most_common)

In [707]:
ped2.loc[ped2.SpeedLimit.isnull(), 'SpeedLimit'] = ped2.RdClass.map(most_common)

In [708]:
ped2.SpeedLimit.isnull().value_counts()

False    33706
Name: SpeedLimit, dtype: int64

In [709]:
ped2.TraffCntrl.value_counts()

No Control Present                     23023
Stop And Go Signal                      4276
Double Yellow Line, No Passing Zone     3280
Stop Sign                               2153
Human Control                            336
Other                                    267
Yield Sign                               100
Flashing Stop And Go Signal               85
Warning Sign                              49
Flashing Signal Without Stop Sign         43
Flashing Signal With Stop Sign            42
School Zone Signs                         42
RR Gate And Flasher                        8
RR Crossbucks Only                         1
RR Flasher                                 1
Name: TraffCntrl, dtype: int64

In [710]:
ped2.TraffCntrl.isnull().value_counts()

False    33706
Name: TraffCntrl, dtype: int64

In [711]:
ped2.loc[ped2['TraffCntrl'] == 'No Control Present', 'TraffCntrl'] = 0
ped2.loc[~(ped2['TraffCntrl'] == 0), 'TraffCntrl'] = 1

In [712]:
ped2.TraffCntrl.value_counts()

0    23023
1    10683
Name: TraffCntrl, dtype: int64

In [713]:
# Snow/Sleet seems more hazardous than fog/smoke and other is pretty small. Would like to be able to combine them all,
# but may be inadvisable because I have no idea what's in other.
ped2.Weather.value_counts()

Clear                                       26037
Cloudy                                       4606
Rain                                         2618
Snow, Sleet, Hail, Freezing Rain/Drizzle      207
Fog, Smog, Smoke                              155
Other                                          83
Name: Weather, dtype: int64

In [714]:
ped2.Weather.isnull().value_counts()

False    33706
Name: Weather, dtype: int64

In [715]:
ped2.DrvrAgeGrp.value_counts()

30-39    4910
40-49    4497
50-59    4123
20-24    3467
25-29    3033
60-69    3032
70+      2451
0-19     2115
Name: DrvrAgeGrp, dtype: int64

In [716]:
ped2.DrvrAgeGrp = ped2.DrvrAgeGrp.fillna('Unknown')

In [717]:
ped2.PedAlcDrg.value_counts()

No                                             21575
Yes-Alcohol, impairment suspected               2503
Yes-Alcohol and Drugs, impairment suspected      203
Yes-Alcohol, impairment detected                 190
Yes-Drugs, impairment suspected                  157
Yes-Alcohol and Drugs, impairment detected        14
Yes-Drugs, impairment detected                    10
Name: PedAlcDrg, dtype: int64

In [718]:
ped2.loc[ped2.PedAlcDrg == 'No', 'PedAlcDrg'] = 0
ped2.loc[ped2.PedAlcDrg != 0, 'PedAlcDrg'] = 1
ped2.PedAlcDrg.value_counts()

0    21575
1    12131
Name: PedAlcDrg, dtype: int64

### Dealing with the Binaries

In [719]:
ped2.loc[ped2.AmbulanceR == 'No', 'AmbulanceR'] = 0
ped2.loc[ped2.AmbulanceR != 0, 'AmbulanceR'] = 1
ped2.AmbulanceR.value_counts()

1    24223
0     9483
Name: AmbulanceR, dtype: int64

In [720]:
ped2.HitRun.value_counts()

No     26947
Yes     6759
Name: HitRun, dtype: int64

In [721]:
ped2.loc[ped2.HitRun == 'No', 'HitRun'] = 0
ped2.loc[ped2.HitRun == 'Yes', 'HitRun'] = 1
ped2.HitRun.value_counts()

0    26947
1     6759
Name: HitRun, dtype: int64

In [722]:
ped2.DrvrAlcDrg.value_counts()

No                                             19825
Yes-Alcohol, impairment suspected                647
Yes-Alcohol, impairment detected                 114
Yes-Drugs, impairment suspected                   90
Yes-Alcohol and Drugs, impairment suspected       74
Yes-Drugs, impairment detected                    18
Yes-Alcohol and Drugs, impairment detected        11
Name: DrvrAlcDrg, dtype: int64

In [723]:
ped2.loc[ped2.DrvrAlcDrg  == 'No', 'DrvrAlcDrg'] = 0
ped2.loc[ped2.DrvrAlcDrg  != 'No', 'DrvrAlcDrg'] = 1
ped2.DrvrAlcDrg.value_counts()

1    33706
Name: DrvrAlcDrg, dtype: int64

In [724]:
ped2.DrvrAlcDrg = ped2.DrvrAlcDrg.fillna('Unknown')

In [725]:
ped2.DrvrInjury.value_counts()

O: No Injury                   25830
C: Possible Injury               875
B: Suspected Minor Injury        402
A: Suspected Serious Injury       32
K: Killed                         13
Name: DrvrInjury, dtype: int64

In [726]:
ped2.DrvrInjury = ped2.DrvrInjury.fillna('Unknown')
print(ped2.DrvrInjury.isnull().sum())
ped2.DrvrInjury.value_counts()

0


O: No Injury                   25830
Unknown                         6554
C: Possible Injury               875
B: Suspected Minor Injury        402
A: Suspected Serious Injury       32
K: Killed                         13
Name: DrvrInjury, dtype: int64

In [727]:
ped2.CrashSevr.value_counts()

C: Possible Injury             13802
B: Suspected Minor Injury      12100
O: No Injury                    2781
A: Suspected Serious Injury     2474
K: Killed                       2188
Name: CrashSevr, dtype: int64

In [728]:
ped2.PedAlcDrg.value_counts()

0    21575
1    12131
Name: PedAlcDrg, dtype: int64

### Checking/Verifying Types

In [729]:
ped2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33706 entries, 0 to 33706
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        33706 non-null  object 
 1   AmbulanceR  33706 non-null  object 
 2   DrvrAlcDrg  33706 non-null  int64  
 3   CrashMonth  33706 non-null  int64  
 4   CrashDay    33705 non-null  float64
 5   CrashHour   33706 non-null  float64
 6   CrashSevr   33345 non-null  object 
 7   DrvrAgeGrp  33706 non-null  object 
 8   DrvrInjury  33706 non-null  object 
 9   DrvrVehTyp  33706 non-null  object 
 10  HitRun      33706 non-null  object 
 11  LightCond   33706 non-null  object 
 12  PedAgeGrp   33706 non-null  object 
 13  PedAlcDrg   33706 non-null  object 
 14  RdClass     33706 non-null  object 
 15  SpeedLimit  33706 non-null  object 
 16  TraffCntrl  33706 non-null  object 
 17  Weather     33706 non-null  object 
dtypes: float64(2), int64(2), object(14)
memory usage: 6.1+ MB


In [730]:
for column in ped2:
    try:
        ped2[column] = pd.to_numeric(ped2[column])
    except ValueError:
        pass

ped2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33706 entries, 0 to 33706
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        33706 non-null  object 
 1   AmbulanceR  33706 non-null  int64  
 2   DrvrAlcDrg  33706 non-null  int64  
 3   CrashMonth  33706 non-null  int64  
 4   CrashDay    33705 non-null  float64
 5   CrashHour   33706 non-null  float64
 6   CrashSevr   33345 non-null  object 
 7   DrvrAgeGrp  33706 non-null  object 
 8   DrvrInjury  33706 non-null  object 
 9   DrvrVehTyp  33706 non-null  object 
 10  HitRun      33706 non-null  int64  
 11  LightCond   33706 non-null  object 
 12  PedAgeGrp   33706 non-null  object 
 13  PedAlcDrg   33706 non-null  int64  
 14  RdClass     33706 non-null  object 
 15  SpeedLimit  33706 non-null  object 
 16  TraffCntrl  33706 non-null  int64  
 17  Weather     33706 non-null  object 
dtypes: float64(2), int64(6), object(10)
memory usage: 6.1+ MB


In [731]:
missing_labels = ped2[ped2.CrashSevr.isnull()].copy()

In [732]:
# We're not missing that many entries, but it is a bit concerning that 75% of the missing Crash Severities
# are hit and runs. Shouldn't the pedestrian be able to describe whether or not they were injured? Maybe these
# accidents were reported by a third-party witness and the two parties involved in the accident fled?
missing_labels.describe().T

,count,mean,std,min,25%,50%,75%,max
AmbulanceR,361.0,0.349030,0.477325,0.0,0.0,0.0,1.0,1.0
DrvrAlcDrg,361.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
CrashMonth,361.0,6.664820,3.364208,1.0,4.0,7.0,9.0,12.0
CrashDay,361.0,4.202216,1.952376,1.0,2.0,4.0,6.0,7.0
CrashHour,361.0,13.606648,6.372280,0.0,10.0,15.0,18.0,23.0
HitRun,361.0,0.753463,0.431593,0.0,1.0,1.0,1.0,1.0
PedAlcDrg,361.0,0.407202,0.491995,0.0,0.0,0.0,1.0,1.0
TraffCntrl,361.0,0.235457,0.424873,0.0,0.0,0.0,0.0,1.0


In [733]:
# How do the stats of the missing values differ from the data at large?
# The missing values happen a little earlier in the year (New Years?), happen earler in the day
# (perhaps from crashes at 1-4 AM?), and have a significantly higher rate of hit and runs.
(missing_labels.describe().T - ped2.describe().T).iloc[:,1:]

,mean,std,min,25%,50%,75%,max
AmbulanceR,-0.369625,0.027663,0.0,0.0,-1.0,0.0,0.0
DrvrAlcDrg,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
CrashMonth,-0.167999,-0.133997,0.0,0.0,0.0,-1.0,0.0
CrashDay,0.027227,0.018645,0.0,-1.0,0.0,0.0,0.0
CrashHour,-0.571777,0.438319,0.0,0.0,0.0,-1.0,0.0
HitRun,0.552935,0.031192,0.0,1.0,1.0,1.0,0.0
PedAlcDrg,0.047296,0.012015,0.0,0.0,0.0,0.0,0.0
TraffCntrl,-0.081489,-0.040420,0.0,0.0,0.0,-1.0,0.0


In [734]:
ped2.HitRun.value_counts()

0    26947
1     6759
Name: HitRun, dtype: int64

(GMC) While the missing values definitely have a larger share of Hit and Runs than the no missing values, since there are only 361 missing values and there are over 6000 recorded hit and runs in the labeled set, I will toss out all the missing values, but will also keep that I did this in mind in case something unusual shows up later.

In [735]:
ped2 = ped2.dropna()
ped2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33344 entries, 0 to 33705
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        33344 non-null  object 
 1   AmbulanceR  33344 non-null  int64  
 2   DrvrAlcDrg  33344 non-null  int64  
 3   CrashMonth  33344 non-null  int64  
 4   CrashDay    33344 non-null  float64
 5   CrashHour   33344 non-null  float64
 6   CrashSevr   33344 non-null  object 
 7   DrvrAgeGrp  33344 non-null  object 
 8   DrvrInjury  33344 non-null  object 
 9   DrvrVehTyp  33344 non-null  object 
 10  HitRun      33344 non-null  int64  
 11  LightCond   33344 non-null  object 
 12  PedAgeGrp   33344 non-null  object 
 13  PedAlcDrg   33344 non-null  int64  
 14  RdClass     33344 non-null  object 
 15  SpeedLimit  33344 non-null  object 
 16  TraffCntrl  33344 non-null  int64  
 17  Weather     33344 non-null  object 
dtypes: float64(2), int64(6), object(10)
memory usage: 4.8+ MB


(GMC) lastly, I want to conmvert the city column into a city population and use a min/max scale.

In [736]:
ped_cities = ped2.City.unique()

# Some ped cities are not in my city data set. The ones that are missing all seem to be from small towns. I will
# manually impute a population based on the smaller towns that I do have.
imputed_pop = nc_cities.Population.quantile(0.2)

# I wanted to use a dict comprehension, but can't because of the exceptions
city_pops = {}

for city in ped_cities:
    try:
        city_pops[city] = nc_cities.loc[nc_cities.City ==city, 'Population'].values[0]
    except IndexError:
        city_pops[city] = imputed_pop

ped2['CityPop'] = ped2.City.map(city_pops).astype('int64')

In [737]:
ped2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33344 entries, 0 to 33705
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        33344 non-null  object 
 1   AmbulanceR  33344 non-null  int64  
 2   DrvrAlcDrg  33344 non-null  int64  
 3   CrashMonth  33344 non-null  int64  
 4   CrashDay    33344 non-null  float64
 5   CrashHour   33344 non-null  float64
 6   CrashSevr   33344 non-null  object 
 7   DrvrAgeGrp  33344 non-null  object 
 8   DrvrInjury  33344 non-null  object 
 9   DrvrVehTyp  33344 non-null  object 
 10  HitRun      33344 non-null  int64  
 11  LightCond   33344 non-null  object 
 12  PedAgeGrp   33344 non-null  object 
 13  PedAlcDrg   33344 non-null  int64  
 14  RdClass     33344 non-null  object 
 15  SpeedLimit  33344 non-null  object 
 16  TraffCntrl  33344 non-null  int64  
 17  Weather     33344 non-null  object 
 18  CityPop     33344 non-null  int64  
dtypes: float64(2), int64(7), 

### Encoding

In [738]:
X = ped2.copy()
y = X.pop('CrashSevr')
X.pop('City')
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33344 entries, 0 to 33705
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AmbulanceR  33344 non-null  int64  
 1   DrvrAlcDrg  33344 non-null  int64  
 2   CrashMonth  33344 non-null  int64  
 3   CrashDay    33344 non-null  float64
 4   CrashHour   33344 non-null  float64
 5   DrvrAgeGrp  33344 non-null  object 
 6   DrvrInjury  33344 non-null  object 
 7   DrvrVehTyp  33344 non-null  object 
 8   HitRun      33344 non-null  int64  
 9   LightCond   33344 non-null  object 
 10  PedAgeGrp   33344 non-null  object 
 11  PedAlcDrg   33344 non-null  int64  
 12  RdClass     33344 non-null  object 
 13  SpeedLimit  33344 non-null  object 
 14  TraffCntrl  33344 non-null  int64  
 15  Weather     33344 non-null  object 
 16  CityPop     33344 non-null  int64  
dtypes: float64(2), int64(7), object(8)
memory usage: 4.6+ MB


In [741]:
print(X.shape)
X = pd.get_dummies(X)
print(X.shape)

(33344, 63)
(33344, 63)


In [747]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33344 entries, 0 to 33705
Data columns (total 63 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   AmbulanceR                                        33344 non-null  int64  
 1   DrvrAlcDrg                                        33344 non-null  int64  
 2   CrashMonth                                        33344 non-null  int64  
 3   CrashDay                                          33344 non-null  float64
 4   CrashHour                                         33344 non-null  float64
 5   HitRun                                            33344 non-null  int64  
 6   PedAlcDrg                                         33344 non-null  int64  
 7   TraffCntrl                                        33344 non-null  int64  
 8   CityPop                                           33344 non-null  int64  
 9   DrvrAgeGrp_0-19  

## Splitting

(GMC) The assignment has me scale the data, but to the best of my knowledge, that should not be done before splitting the data. After I have finished one-hot encoding my data, I will split it and scale the appropriate feature (CityPop)

In [744]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [746]:
scaler = MinMaxScaler()
X_train.CityPop = scaler.fit_transform(X_train[['CityPop']])
X_train.CityPop

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


20349    0.193751
21063    1.000000
15646    0.130113
27167    0.339607
18434    0.018005
           ...   
17038    0.314409
6346     0.541627
11423    1.000000
875      0.000405
15974    1.000000
Name: CityPop, Length: 25008, dtype: float64